In [1]:
using Pkg
Pkg.activate(".")

#=
Pkg.add(
["QuantumOptics","Flux", "ProgressMeter", "Random", 
"Statistics", "SparseArrays", "Zygote", "LinearAlgebra", 
"StatsBase","DiffEqFlux","Optimisers","Compat",
"PlotlyJS","DataFrames", "CSV","BSON"]
)
=#

Pkg.instantiate()
Pkg.status()

#############################################
#= Packages =#
using Flux, ProgressMeter, Random, Statistics, QuantumOptics, SparseArrays, StatsBase, LinearAlgebra
using Zygote, DifferentialEquations, SciMLSensitivity, DiffEqFlux, Optimisers, Compat, PlotlyJS, CSV, DataFrames, BSON
############################################

  Activating project at `c:\Users\dstir\OneDrive - USN\Fock ladder\Correcting`


Status `C:\Users\dstir\OneDrive - USN\Fock ladder\Correcting\Project.toml`
  [fbb218c0] BSON v0.3.9
  [336ed68f] CSV v0.10.15
  [34da2185] Compat v4.16.0
  [a93c6f00] DataFrames v1.7.0
  [aae7a2af] DiffEqFlux v4.3.0
  [0c46a032] DifferentialEquations v7.16.1
⌃ [587475ba] Flux v0.16.3
  [3bd65402] Optimisers v0.4.6
  [f0f68f2c] PlotlyJS v0.18.16
  [92933f4c] ProgressMeter v1.10.4
  [6e0679c1] QuantumOptics v1.2.3
⌃ [1ed8b502] SciMLSensitivity v7.79.0
  [2913bbd2] StatsBase v0.34.5
⌃ [e88e6eb3] Zygote v0.7.7
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [2f01184e] SparseArrays v1.10.0
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


WebIO._IJuliaInit()

In [ ]:
################################################
  #= PRELIMINARY DEFINITIONS AND DYNAMICS =#
################################################
include("HBAR-qubit.jl")
include("QM_library.jl")
include("ML_NN_library.jl")

N_mech = 3
qub, mech, HBAR_qubit = Qubit_HO(N_mech-1, :FockBasis, 1//2)

ψ0 = tensor(spindown(qub.basis), fockstate(mech.basis, 0));
p = [1e3, 1e3];

Iad = HBAR_qubit.Iad.data
mI = HBAR_qubit.mI.data
Ia = HBAR_qubit.Ia.data
pI = HBAR_qubit.pI.data
zI = HBAR_qubit.zI.data
xI = HBAR_qubit.xI.data

function to_real_vec(vector)
    real_imag = Vector{Float64}(undef, 2 * length(vector))
    for i in eachindex(vector)
        real_imag[2i - 1] = real(vector[i])
        real_imag[2i]     = imag(vector[i])
    end
    return real_imag
end

function SE_Fock_dynamics(du, u, p, t) #
    Δ, Ω = p

    #= Hamiltonians of the problem =#
    H_JC = g * (Iad * mI  + Ia * pI)
    H0 = (Δ / 2) * zI
    H_drive = Ω * xI

    H_tot = H0 + H_JC + H_drive

    ψ = u[1:2:end] + im * u[2:2:end]
    dψ = -1im * H_tot * ψ

    for i in 1:length(dψ)
        du[2i-1] = real(dψ[i]) 
        du[2i] = imag(dψ[i])
    end

end

function SE_Fock_problem!(p)
    return ODEProblem(SE_Fock_dynamics, to_real_vec(ψ0.data), (0.0, 1.0), p)
end



function Quantum_solver_ODE(prob)
    sol = solve(prob, Tsit5())

    return sol.t, sol.u

end

prob = SE_Fock_problem!(p)
tspan, sol = Quantum_solver_ODE(prob)

ψ_final = recomposition(sol[end])

to_real_vec (generic function with 1 method)